# Extraction of electrical features (eFeatures) from experimental data

____

## Overview

____

In this tutorial we will see how to extract electrical features (eFeatures), such as spike amplitude, firing frequency, etc... from experimental traces. The eFeatures describe the electrical behavior our neuron model should reproduce.

The steps we will follow are:

* Select and visualize the data.

* Electrophysiological features will be extracted from the voltage traces, thanks to the ** Electrophys Feature Extraction Library ** [eFEL](https://github.com/BlueBrain/eFEL).

* We will use experimental current traces to create protocols that we will use to simulate our neuron model.

* In future weeks we will use the **Blue Brain Python Optimisation Library** [BluePyOpt](https://github.com/BlueBrain/BluePyOpt) to create a model template for the [NEURON simulator](https://www.neuron.yale.edu/neuron/). There you'll see how the morphology you've chosen, the eFeatures and the stimuli will be combined in setting up the optimization of your neuron model.

In [ ]:
# First install the pacakges you will need.
! pip install --upgrade pip
! pip install json2html
# First execute this line to upgrade pip
! pip install -q --upgrade "hbp-service-client==1.0.0"

__RESTART YOUR KERNEL NOW!__

Now, we can import some useful Python modules.

In [ ]:
%load_ext autoreload
%autoreload

import numpy, IPython
import json, os

import matplotlib.pyplot as plt
%matplotlib notebook

import collections

from json2html import *

# 1. Electrophysiology data
In this section we will process the electrophysiological data recorded with patch clamp (current clamp) experiments.

Please, go to the __MOODLE__ and download the __.dat__ files. Then, save these files in the __Storage__ of your collab. There are a lot of files (exactly 152), so is better to create a folder in the Storage and save the files in the folder. I called the folder __"week6_data"__

In [ ]:
# Download the files from the Storage to your collab virtual machine
# Save the file's names in a list
# Be patient it could take some minutes...

# These lines allow us to take and work with the desired files in the storage.
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
data_dir = collab_path + '/week6_data/'

#print files_list
project_uuid = clients.storage.api_client.get_entity_by_query(path=data_dir)['uuid']
l = clients.storage.api_client.list_folder_content(project_uuid, page_size=200)

filenames = []
for e in l['results']:
    filenames.append(e['name'])

for filename in filenames:
    print filename
    path_to_file = '/tmp/downloaded_'+filename
    file_from_path = data_dir+filename
    clients.storage.download_file(str(data_dir+filename), str(path_to_file))


___
### Traces description

* All the recordings you see above represent different **stimuli** (e.g. "APWaveform", "FirePattern", "IV"). 
* Each stimulus comprises different **sweeps** (e.g. "APWaveform*46-51"), of increasing/decreasing amplitudes.
* Each stimulus is repeated multiple times (e.g. APWaveform 46-51, 1042-1047, 2042-2047, 3042-3047 ). In the example above we have four **repetitions** of each stimulus.

Any individual recording has a trace number (e.g. "_1046"). Note that we have pairs of recordings with the same trace number (e.g. "exp_APWaveform_ch7_51.dat" and "exp_APWaveform_ch6_51.dat"). One of them contains the current stimulus (in this case "*ch7*") and the other the voltage response (in this case "*ch6*").
___

For this example we have chosen to use one negative current step and two positive current steps with different durations. 

We can for example select a sweep from a stimulus (in this case the one with the highest amplitude) and some repetitions.

With the code below we can for example select traces based on trace number and store them in Python dictionaries.

In [ ]:
selected_traces = [1047, 2047, 3047, 1049, 2049, 3049, 1041, 2041, 3041]

# Store voltage data in a dictionary step_name : [list of repetitions]
steps_v_dict = collections.OrderedDict({'LongStepNeg': [], 'ShortStepPos': [], 'LongStepPos': []})

# Store current data in a dictionary step_name : [list of repetitions]
steps_i_dict = collections.OrderedDict({'LongStepNeg': [], 'ShortStepPos': [], 'LongStepPos': []})


for file_name in filenames:
    # Get channel and trace number from the file_name
    channel = int(file_name[:-4].split('_')[2][2:])
    tracenum = int(file_name[:-4].split('_')[-1])
    path_to_file = '/tmp/downloaded_'+file_name
    
    # Even channel numbers are voltage traces in this case
    if channel % 2 == 0:
        if "APWaveform" in file_name and tracenum in selected_traces:
            steps_v_dict['ShortStepPos'].append(numpy.fromfile(path_to_file))
        if "FirePattern" in file_name and tracenum in selected_traces:
            steps_v_dict['LongStepPos'].append(numpy.fromfile(path_to_file))
        if "IV" in file_name and tracenum in selected_traces:
            steps_v_dict['LongStepNeg'].append(numpy.fromfile(path_to_file))
            
    # Odd channel numbers are voltage traces in this case        
    elif channel % 2 == 1:
        if "APWaveform" in file_name and tracenum in selected_traces:
            steps_i_dict['ShortStepPos'].append(numpy.fromfile(path_to_file))
        if "FirePattern" in file_name and tracenum in selected_traces:
            steps_i_dict['LongStepPos'].append(numpy.fromfile(path_to_file))
        if "IV" in file_name and tracenum in selected_traces:
            steps_i_dict['LongStepNeg'].append(numpy.fromfile(path_to_file))


We can now plot these traces.

In [ ]:
# Initialize a figure
fig1, axes = plt.subplots(len(steps_v_dict), sharey = True, figsize=(20,15))

# Plot the voltage traces
for idx, step_name in enumerate(steps_v_dict.keys()):
    for rep, trace in enumerate(steps_v_dict[step_name]):
        data = trace.reshape(len(trace)/2,2)
        axes[idx].plot(data[:,0],data[:,1], label = 'Rep. ' + str(rep+1))
        axes[idx].set_ylabel('Voltage (mV)')
        axes[idx].legend(loc = 'best')
        axes[idx].set_title(step_name)
    axes[-1].set_xlabel('Time (ms)')

# 2. Electrophysiological features
To build a detailed neuron model, we need to quantify the electrical behavior we want to reproduce. The metrics we use are the eFeatures, that measure parameters describing for instance the shape of the action potential or the firing properties of a neuron (see [here](http://bluebrain.github.io/eFEL/eFeatures.html) for eFeatures description).

In this particular example, we extract distinct features from the three types of voltage traces.
The eFeatures extracted from the data and later from the model will be used to evaluate the results of the simulations. The mean features values, along with the standard deviations will be stored in a .json file.

In [ ]:
# Extract features
import efel

# Define stimulus start and end times
steps_info = {'LongStepNeg': [250, 3250], 'ShortStepPos': [250, 475], 'LongStepPos': [250, 3850]}

# Prepare the traces for eFEL
def get_features(data):
    # All the traces converted in eFEL format
    efel_traces = {'LongStepNeg': [], 'ShortStepPos': [], 'LongStepPos': []}
    for step_name, step_traces in data.items():
        for rep in step_traces:            
            data = rep.reshape(len(rep)/2,2)
            # A single eFEL trace 
            trace = {}
            trace['T'] = data[:,0]
            trace['V'] = data[:,1] 
            trace['stim_start'] = [steps_info[step_name][0]]
            trace['stim_end'] = [steps_info[step_name][1]]
            trace['name'] = step_name
            
            efel_traces[step_name].append(trace)
    
    features_values = collections.defaultdict(dict)       
    
    features_values['LongStepNeg'] = efel.getMeanFeatureValues(efel_traces['LongStepNeg'], 
                                                                ['time_constant', 'voltage_deflection_begin', 
                                                                'voltage_deflection'])
    
    features_values['LongStepPos'] = efel.getMeanFeatureValues(efel_traces['LongStepPos'], 
                                                               ['mean_frequency', 'adaptation_index2', 
                                                                'ISI_CV', 'doublet_ISI'])
    
    features_values['ShortStepPos'] = efel.getMeanFeatureValues(efel_traces['ShortStepPos'], 
                                                                ['time_to_first_spike', 'AHP_depth', 
                                                                'AP_width', 'AP_height'])    

    return features_values

We can now visualise the feature values we computed, each row in the table corresponds to a repetition of the same step.

In [ ]:
efel_features = dict(get_features(steps_v_dict))
IPython.display.HTML(json2html.convert(json=efel_features))

We compute features mean and standard deviation.

In [ ]:
features_dict = collections.OrderedDict()
for step_name, reps in efel_features.items():
    feature_values = collections.defaultdict(list)
    for rep in reps: 
        for feature_name, value in rep.iteritems():
            feature_values[feature_name].append(value)
   
    features_dict[step_name] = {"soma":{}}
    for name, values in feature_values.items():
        features_dict[step_name]["soma"][name] = [numpy.mean(values), numpy.std(values)]
        
IPython.display.HTML(json2html.convert(json=dict(features_dict)))

We write the eFeatures in a json file that we will use later in the exercise.

In [ ]:
with open('features.json', 'w') as fp:
    json.dump(features_dict, fp, indent = 4)

In [ ]:
# Let's see if we had created the file in the working directory of the collab virtual machine.
! ls

In [ ]:
# Execute the lines below
# This will save the files in the storage
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
clients.storage.upload_file('features.json', data_dir + '/features.json', 'application/JSON')

___
### Exercise 1 - Plot some spike eFeatures 

Complete the code below in order to extract the 1. action potentials (AP) peak times, 2. their height (maximum overshoot voltage) and 3. the minimum voltage between the spikes of the "LongStepPos" traces and plot them.

The result should be similar to the last figures in this [eFEL example](https://github.com/BlueBrain/eFEL/blob/master/examples/nmc-portal/L5TTPC2.ipynb).
___

In [ ]:
# Retrive the LongStepPos traces from the steps_dict dictionary
traces_data = ___

traces_efel = []

for idx in range(len(traces_data)):
    trace = {}
    data = traces_data[idx].reshape(len(traces_data[idx])/2,2)
    trace['T'] = data[:,0]
    trace['V'] = data[:,1]
    trace['stim_start'] = [steps_info['LongStepPos'][0]]
    trace['stim_end'] = [steps_info['LongStepPos'][1]]
    
    traces_efel.append(trace)

In [ ]:
# Use the efel "getFeatureNames" function to find the names
# for the peak times, action potential height and the the AHP absolute depth

print ___

In [ ]:
efeatures = efel.getFeatureValues(traces_efel, [____]) # Insert here three feature names
fig1, axes = plt.subplots(len(traces_data))

# These are "list comprehensions", a more compat way for writing for loops in Python
[axes[rep].plot(traces_efel[rep]['T'], traces_efel[rep]['V']) for rep in range(len(traces_data))]

# Take inspiration from the line above to plot spike times (x axis) and spike hights (y axis)
[___ for rep in range(len(traces_data))]

# Take inspiration from the line above to plot spike times (x axis) and AHP depths (y axis)
[___ for rep in range(len(traces_data))]

## 3. Write out the stimulation protocols

Now it's time to process the current stimuli that were used to record the voltage responses seen above.

We will estimate the stimuli amplitude from the trace and save them in a file "protocols.json". They will be used later on in the project to stimulate your neuron model.

In [ ]:
# Plot the current traces
# Initialize a figure
fig1, axes = plt.subplots(len(steps_i_dict), sharey = True, figsize=(20,15))

for idx, step_name in enumerate(steps_i_dict.keys()):
    for rep, trace in enumerate(steps_i_dict[step_name]):
        data = trace.reshape(len(trace)/2,2)
        axes[idx].plot(data[:,0],data[:,1], label = 'Rep. ' + str(rep+1))
        axes[idx].set_ylabel('Current (nA)')
        axes[idx].legend(loc = 'best')
        axes[idx].set_title(step_name)
    axes[-1].set_xlabel('Time (ms)')

In [ ]:
protocols_dict = collections.OrderedDict()

# Stimuli start and end time
steps_info = {'LongStepNeg': [250, 3250], 'ShortStepPos': [250, 475], 'LongStepPos': [250, 3850]}

# Stimuli holding current and step current amplitudes in nA
amps_info = collections.defaultdict(list)
for step_name in steps_i_dict.keys():
    
    iholds = []
    isteps = []
    for trace in steps_i_dict[step_name]:
        data = trace.reshape(len(trace)/2,2)
        tot_duration = steps_info[step_name][1]+steps_info[step_name][0]
   
        dt = float(tot_duration)/len(data)
        ihold = numpy.mean(data[:,1][0:int(steps_info[step_name][0]/dt)])

        istep = numpy.mean(data[:,1][int(steps_info[step_name][0]/dt):int(steps_info[step_name][1]/dt)])-ihold
        iholds.append(ihold)
        isteps.append(istep)
       
    amps_info[step_name].append(round(numpy.mean(isteps), 4))
    amps_info[step_name].append(round(numpy.mean(iholds), 4)) 
    
#amps_info  = {'LongStepNeg': [-0.01, 0.05], 'ShortStepPos': [0.18,0.05],'LongStepPos': [0.15 ,0.05]}

for step_name, reps in efel_features.items():   
    protocols_dict[step_name] = {"stimuli":[]}
    protocols_dict[step_name]["stimuli"].append({"delay":steps_info[step_name][0],
                                               "amp":amps_info[step_name][0],
                                               "duration":steps_info[step_name][1]-steps_info[step_name][0],
                                               "totduration":steps_info[step_name][1]+steps_info[step_name][0]})
    protocols_dict[step_name]["stimuli"].append({"delay":0,
                                               "amp":amps_info[step_name][1],
                                               "duration":steps_info[step_name][1]+steps_info[step_name][0],
                                               "totduration":steps_info[step_name][1]+steps_info[step_name][0]})
    
IPython.display.HTML(json2html.convert(json=dict(protocols_dict)))


In [ ]:
# Save the protocols in a .json file
with open('protocols.json', 'w') as fp:
    json.dump(protocols_dict, fp, indent = 4)

In [ ]:
# Execute the lines below
# This will save the file in the storage
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
clients.storage.upload_file('protocols.json', data_dir + '/protocols.json', 'application/JSON')